# 벡터자기회귀(VAR) 모형

----------
# 0. 환경설정

In [1]:
# 라이브러리 호출
import numpy as np
import pandas as pd
import time
import glob
import pickle
import itertools

import statsmodels.api as sm
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# 데이터프레임 출력 옵션
pd.set_option('display.max_columns', 100)

#지수표현
pd.options.display.float_format = '{:.5f}'.format

# # 그래프 폰트
# font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
# rc('font', family=font_name)
# plt.rc('font', family='Malgun Gothic')
# plt.rcParams["figure.figsize"] = (8, 4)
# plt.rcParams['axes.unicode_minus'] = False

----------
# 1. 입력값 기입

## 1-1. 표준화 정보

In [2]:
## 독립변수(x)에 대하여 표준화 진행 시 원하는 표준화 기법 선택 (입력 안하면 표준화 진행 X)
# 1 : 표준화1(StandardScaler) : 평균 = 0 / 표준편차 = 1
# 2 : 표준화2(Normalization) : MinMaxScaler : 최소값 0 ~ 최대값 1 : 이상값에 영향 받음
# 3 : 표준화3(RobustScaler) : 중앙값 = 0 / IQR(1분위(25%) ~ 3분위(75%)) = 1 
select_scaler = 1  # (1, 2, 3) 중 택 1

## 1-2. 상관분석 정보

In [3]:
## 상관계수 값 설정 (y와 x변수간의 단일 상관계수 값이 'corr_stand_val' 이상인 x변수만 입력변수로 사용)
# 입력변수 : 모델링에 이용되는 변수
corr_stand_val = 0.5

----
# 2. 데이터 호출

In [4]:
# 데이터 호출 및 필요 컬럼 정리
data = pd.read_csv('data/ts_data(jena_climate_2009_2016).csv', encoding = 'utf-8')
data.columns = list(pd.Series(list(data.columns)).map(lambda x : x.replace(' ','')))

# (날짜변수/종속변수/독립변수) 컬럼명 정의 : x_colnm
tm_colnm = 'DateTime'
y_colnm = 'T(degC)'
x_colnm = list(set(data.columns).difference({y_colnm}).difference({tm_colnm}))


# # 임시 --------------------------------------------------------------------------------------
# use_col = ['DateTime','T(degC)','p(mbar)','rh(%)','VPmax(mbar)','H2OC(mmol/mol)','wv(m/s)']
# data = data[use_col]
data = pd.concat([data[y_colnm].shift(-1), data[x_colnm]], axis = 1)
data = data.iloc[:1000]

In [5]:
for i in data.columns:
    print(type(data[i][0]))

<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>


------
# 3. 모델링

## 3-1. 데이터 전처리

In [6]:
# -------------------------------------------------------------------------------------------------------------- #
# 데이터 형 변환(str -> float)
data[[y_colnm] + x_colnm] = data[[y_colnm] + x_colnm].astype('float')
# -------------------------------------------------------------------------------------------------------------- #
# (train_x / train_y / test_x / test_y) 데이터 분할
train_x, test_x, train_y, test_y = train_test_split(data[x_colnm], data[y_colnm], test_size = 0.2, shuffle = False, random_state = 1234)
train_x.reset_index(drop=True, inplace=True)
test_x.reset_index(drop=True, inplace=True)
train_y.reset_index(drop=True, inplace=True)
test_y.reset_index(drop=True, inplace=True)
# -------------------------------------------------------------------------------------------------------------- #
## 표준화 수행

# StandardScaler : 평균 0, 표준편차 1
if select_scaler == 1:
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()   
    mody_train_x = pd.DataFrame(scaler.fit_transform(train_x), columns = list(train_x.columns))
    mody_test_x = pd.DataFrame(scaler.transform(test_x), columns = list(test_x.columns))

# Normalization : MinMaxScaler : 최소값 0 ~ 최대값 1
elif select_scaler == 2:
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    mody_train_x = pd.DataFrame(scaler.fit_transform(train_x), columns = list(train_x.columns))
    mody_test_x = pd.DataFrame(scaler.transform(test_x), columns = list(test_x.columns))

# RobustScaler : 중앙값 0, IQR(1분위(25%) ~ 3분위(75%)) 1 : 이상치(outlier) 영향 최소화 / 더 넓게 분포
elif select_scaler == 3:
    from sklearn.preprocessing import RobustScaler
    scaler = RobustScaler()
    mody_train_x = pd.DataFrame(scaler.fit_transform(train_x), columns = list(train_x.columns))
    mody_test_x = pd.DataFrame(scaler.transform(test_x), columns = list(test_x.columns))

# 표준화 수행 X
else:
    mody_train_x = train_x
    mody_test_x = test_x
# -------------------------------------------------------------------------------------------------------------- #
# inf / -inf 값을 null 처리
mody_train_x = mody_train_x.replace([np.inf, -np.inf], np.nan)
mody_test_x = mody_test_x.replace([np.inf, -np.inf], np.nan)        
# -------------------------------------------------------------------------------------------------------------- #
# 모델에 사용할 train, test 데이터셋
mdl_train_data = pd.concat([train_y, mody_train_x], axis = 1)
mdl_test_data = mody_test_x
# -------------------------------------------------------------------------------------------------------------- #
## 독립변수 선택

# 1. 상관분석 : 상관분석은 train 데이터셋에 대해서만 진행 (test 데이터셋 이용 X)
corr = mdl_train_data.corr(method = 'pearson')  # default는 'pearson'
corr = corr.reset_index().rename(columns = {'index':'COLNM'})
corr = corr.loc[corr['COLNM'] != y_colnm,]
corr = corr[corr[y_colnm] >= corr_stand_val]

# 2. 상관분석 결과로 선택된 독립변수 목록
mdl_x_colnm = list(corr['COLNM'])

# 3. 독립변수 목록 중 null값이 없는 독립변수만 선택
train_na_col = []
for col in mdl_test_data.columns:
    if len(mdl_train_data.loc[mdl_train_data[col].isna(),]) != 0:
        train_na_col.append(col)

test_na_col = []
for col in mdl_test_data.columns:
    if len(mdl_test_data.loc[mdl_test_data[col].isna(),]) != 0:
        test_na_col.append(col)

tot_na_col = list(set(train_na_col + test_na_col))  # null값이 있는 독립변수들
mdl_x_colnm = list(set(mdl_x_colnm).difference(set(tot_na_col)))  # 모델에 사용할 독립변수들(상관분석 결과 - null값이 있는 변수)

if len(mdl_x_colnm) == 0:  # 모델에 사용할 독립변수의 수가 0이면 => 기본 독립변수 중 null값이 없는 변수를 선택 
    mdl_x_colnm = x_colnm
    mdl_x_colnm = list(set(mdl_x_colnm).difference(set(tot_na_col)))  # 모델에 사용할 독립변수들
# -------------------------------------------------------------------------------------------------------------- #

mdl_train_data = mdl_train_data[[y_colnm] + mdl_x_colnm]
mdl_test_data = mdl_test_data[mdl_x_colnm]

## 3-2. 정상성 확인 (ADF-Test)

In [ ]:
## 정상성 확인 (ADF-Test)
# 귀무가설 : 정상성이 없다 = 시계열 흐름이 불규칙하다
# p-value가 0.05보다 크면 귀무가설을 기각할 수 없음 = 정상성이 없다 = 시계열 흐름이 불규칙하다
ADF_test_statistic = []
p_value = []
for i in mdl_train_data.columns[1:]:
    adfuller_test = adfuller(mdl_train_data[i], autolag= "AIC")
    ADF_test_statistic.append(adfuller_test[0])
    p_value.append(adfuller_test[1])

ADF_col_nm = pd.DataFrame(mdl_train_data.columns[1:]).rename(columns = {0:'col_nm'})
ADF_test_statistic = pd.DataFrame(ADF_test_statistic).rename(columns = {0:'ADF_test_statistic'})
p_value = pd.DataFrame(p_value).rename(columns = {0:'p_value'})

ADF_test = pd.concat([ADF_col_nm, ADF_test_statistic, p_value], axis = 1)

# 전체 ADF_test 결과 출력
ADF_test

In [39]:
ADF_test

,col_nm,ADF_test_statistic,p_value
0,VPact(mbar),-0.93889,0.77490
1,Tdew(degC),-0.94636,0.77232
2,H2OC(mmol/mol),-0.94971,0.77116
3,Tpot(K),-1.77611,0.39239
4,sh(g/kg),-0.94497,0.77280
5,VPmax(mbar),-1.46610,0.55021


In [44]:
if len(list(ADF_test.loc[ADF_test['p_value'] < 0.05,'col_nm'])) == 0:
    pass
else:
    sdsad

0

In [47]:
 = list(ADF_test.loc[ADF_test['p_value'] >= 0.05,'col_nm'])

['VPact(mbar)',
 'Tdew(degC)',
 'H2OC(mmol/mol)',
 'Tpot(K)',
 'sh(g/kg)',
 'VPmax(mbar)']

## 3-3. VAR 모형 생성

In [8]:
## VAR 모형의 최적 순서 탐색 : (1) AIC 값은 낮을수록 좋음 / (2) Grid-Search를 수행하여 최소 AIC를 만족하는 최적의 p를 탐색
model = VAR(mdl_train_data)

fit_param = []
result_aic = []
for p in range(1,len(mdl_train_data) + 1):
    try :
        fitted_model = model.fit(p)  # 모델 생성
        fit_param.append(p)
        result_aic.append(fitted_model.aic)
    except :
        pass

fit_param = pd.DataFrame(fit_param, columns = ['fit_param'])
result_aic = pd.DataFrame(result_aic, columns = ['result_aic'])
fit_param_df = pd.concat([fit_param,result_aic], axis = 1)

# 최적의 p 값 도출 : fit_param_val
fit_param_val = list(fit_param_df.loc[fit_param_df['result_aic'] == fit_param_df['result_aic'].min(), 'fit_param'])[0]

# # 최적 p 값 그래프로 확인
# plt.plot(fit_param_df['fit_param'], fit_param_df['result_aic'])
# plt.plot(fit_param_df['fit_param'][0:200], fit_param_df['result_aic'][0:200])

In [9]:
## 모델 생성
fitted_model = model.fit(fit_param_val)
# fitted_model.summary()  # 모델 fitting 결과

In [10]:
## 예측 수행
forecast_num = len(mdl_test_data)  # 몇 개의 값을 예측할 것인지?

In [38]:
lagged_values = np.array(mdl_train_data[-fit_param_val:])  # mdl_train_data[-fit_param_val:].values = np.array(mdl_train_data[-fit_param_val:])
predict = pd.DataFrame(fitted_model.forecast(lagged_values, steps = forecast_num), columns = list(mdl_train_data.columns))
predict

,T(degC),VPact(mbar),Tdew(degC),H2OC(mmol/mol),Tpot(K),sh(g/kg),VPmax(mbar)
0,-10.53819,-1.68107,-1.80017,-1.63358,-1.33837,-1.64027,-1.52837
1,-10.52668,-1.54599,-1.61314,-1.50313,-1.19735,-1.51891,-1.47888
2,-10.56718,-1.43435,-1.52780,-1.39268,-1.19111,-1.41533,-1.44647
3,-10.56739,-1.43657,-1.52899,-1.37209,-1.19164,-1.42588,-1.43529
4,-10.19992,-1.41024,-1.46219,-1.35647,-1.18398,-1.44582,-1.43978
...,...,...,...,...,...,...,...
195,2034.07616,-251.71356,-572.72463,-219.14309,-784.08747,-207.66427,-449.21199
196,2827.04024,117.46556,450.21273,204.79874,455.16762,-9.27966,307.56308
197,-3423.92988,273.12524,497.97121,206.01338,658.20656,266.88268,354.45539
198,-2161.49288,-188.06106,-647.24893,-262.64085,-778.92847,-47.45427,-496.14497


In [ ]:
# -------------------------------------------------------------------------------------------------------------- #
## 결과값 정리
rslt = pd.concat([pred, test_y], axis = 1)
rslt['target'] = trg
rslt['stand_time'] = list(data.loc[~data['YEAR'].isin(train_year),'STAND_TIME'])
rslt['BEST_MDL'] = save_lb['model_id'][0]  # 최적 모델명
rslt['DIFF'] = rslt['PREDICT'] - rslt[y_colnm]
rslt['mdl_x_colnm'] = str(mdl_x_colnm)

rslt['PREDICT'] = round(rslt['PREDICT'],4)
rslt[y_colnm] = round(rslt[y_colnm],4)
rslt['DIFF'] = round(rslt['DIFF'],4)

rslt = rslt[['target', 'stand_time', 'PREDICT', y_colnm, 'DIFF', 'BEST_MDL']]
# -------------------------------------------------------------------------------------------------------------- #
## 결과값 저장
if trg == loop_target[0]:
    fin_result = rslt
else:
    fin_result = fin_result.append(rslt)

fin_result.to_csv(save_folder + '/result_' + file_nm + '.csv', index=False, encoding = 'utf-8')

------------